In [1]:
import os
os.chdir("/content/drive/My Drive/Translator/")

In [2]:
os.getcwd()

'/content/drive/My Drive/Translator'

In [7]:
!git clone https://github.com/pytorch/fairseq.git

fatal: destination path 'fairseq' already exists and is not an empty directory.


In [3]:
!pip install fairseq

In [20]:
!git clone https://github.com/moses-smt/mosesdecoder

fatal: destination path 'mosesdecoder' already exists and is not an empty directory.


In [21]:
!git clone https://github.com/rsennrich/subword-nmt

fatal: destination path 'subword-nmt' already exists and is not an empty directory.


In [34]:
!bash prepare_data.sh

pre-processing train data...
Tokenizer Version 1.1
Language: mr
Number of threads: 8

Tokenizer Version 1.1
Language: en
Number of threads: 8

clean-corpus.perl: processing data.tokenized.mr-en/tmp/train.tags.mr-en.tok.mr & .en to data.tokenized.mr-en/tmp/train.tags.mr-en.clean, cutoff 1-175, ratio 9
...
Input sentences: 33007  Output sentences:  33007
Tokenizer Version 1.1
Language: mr
Number of threads: 8

Tokenizer Version 1.1
Language: en
Number of threads: 8

Tokenizer Version 1.1
Language: mr
Number of threads: 8

Tokenizer Version 1.1
Language: en
Number of threads: 8

learn_bpe.py on data.tokenized.mr-en/tmp/train.mr-en...
apply_bpe.py to train.mr...
apply_bpe.py to valid.mr...
apply_bpe.py to test.mr...
apply_bpe.py to train.en...
apply_bpe.py to valid.en...
apply_bpe.py to test.en...


In [35]:
!fairseq-preprocess --source-lang mr --target-lang en \
    --trainpref data.tokenized.mr-en/train --validpref data.tokenized.mr-en/valid --testpref data.tokenized.mr-en/test \
    --destdir data-bin/data.mr-en \
    --workers 20

2021-02-23 18:49:46 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin/data.mr-en', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='mr', srcdict=None, target_lang='en', task='translation', tensorboard_logdir=None, testpref='data.tokenized.mr-en/test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None,

In [4]:
!pip install sacremoses

In [7]:
!CUDA_VISIBLE_DEVICES=0 fairseq-train \
    data-bin/data.mr-en \
    --arch transformer --share-decoder-input-output-embed \
    --min-loss-scale 0.001\
    --encoder-embed-dim 512\
    --encoder-layers 6\
    --encoder-attention-heads 4\
    --decoder-attention-heads 4\
    --decoder-layers 6\
    --optimizer adam --adam-betas '(0.9, 0.98)' --adam-eps 10e-9 \
    --lr 5e-4 --warmup-init-lr 10e-7 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.1 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-epoch 14\
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "lenpen":1}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --keep-best-checkpoints 10\
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

2021-02-23 19:35:12 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='bleu', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='data-bin/data.mr-en', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=4, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=2048, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disable_validation=False, dist

In [8]:
!fairseq-generate data-bin/data.mr-en \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe


Streaming output truncated to the last 5000 lines.
D-1501	-0.7205454111099243	we &apos;re just student .
P-1501	-0.0892 -0.3110 -0.5300 -3.1605 -0.1235 -0.1091
S-1709	टॉमला जराश ् या झोपेची गरज आहे .
T-1709	tom needs some sleep .
H-1709	-1.297169804573059	tom needs a lot of coffee .
D-1709	-1.297169804573059	tom needs a lot of coffee .
P-1709	-0.0415 -0.8253 -0.3648 -5.1548 -0.0671 -3.7248 -0.0834 -0.1156
S-2080	त ् यांच ् याकडे एक चित ् र आहे .
T-2080	she has a picture .
H-2080	-0.5370382070541382	he has a picture .
D-2080	-0.5370382070541382	he has a picture .
P-2080	-0.3493 -0.1524 -0.1044 -0.7355 -1.7639 -0.1167
S-2098	त ् यांची स ् वप ् ने खरी झाली .
T-2098	their dreams came true .
H-2098	-0.9681568741798401	he came true true .
D-2098	-0.9681568741798401	he came true true .
P-2098	-0.6491 -0.5221 -1.6784 -1.7726 -1.0729 -0.1139
S-2480	डिज ् नीलॅंडची सुरुवात कोणी केलेली ?
T-2480	who started disneyland ?
H-2480	-1.6889543533325195	who was the capital of the money ?
D-2480	-1.6889543